<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Update jobtitle country industry from linkedin
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Hubspot/Hubspot_update_jobtitle_country_industry_from_linkedin.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

#hubspot #crm #sales #linkedin #identity

## Input

### Import library

In [ ]:
from naas_drivers import hubspot, linkedin
import naas
import pandas as pd

### Enter Hubspot api key

In [ ]:
auth_token = "YOUR_HUBSPOT_API_KEY"

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
LI_AT = 'YOUR_COOKIE_LI_AT'  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = 'YOUR_COOKIE_JSESSIONID'  # EXAMPLE ajax:8379907400220387585

### Connect to Hubspot

In [ ]:
hs = hubspot.connect(auth_token)

### Schedule your notebook everyday

In [ ]:
naas.scheduler.add(cron="15 6 * * *")

### Get all contacts in Hubspot

In [ ]:
properties_list = [
    "hs_object_id",
    "firstname",
    "lastname",
    "linkedinbio",
    "jobtitle",
    "country",
    "industry",
]
hubspot_contacts = hs.contacts.get_all(properties_list).fillna("Not Defined")
hubspot_contacts

# Model

### Filter to get "jobtitle", "country", "industry" = "Not Defined" and "linkedinbio" = defined

In [ ]:
df_to_update = hubspot_contacts.copy()

# Filter on "Not defined"
df_to_update = df_to_update[(df_to_update.linkedinbio != "Not Defined") &
                            (df_to_update.jobtitle == "Not Defined") &
                            (df_to_update.country == "Not Defined") & 
                            (df_to_update.industry == "Not Defined")]

# Limit to last 50 contacts
df_to_update = df_to_update.sort_values(by="createdate", ascending=False)[:50].reset_index(drop=True)

df_to_update

### Get identity from Linkedin

In [ ]:
for _, row in df_to_update.iterrows():
    linkedinbio = row.linkedinbio
    
    # Get followers
    df = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(linkedinbio)
    jobtitle = df.loc[0, "OCCUPATION"]
    industry = df.loc[0, "INDUSTRY_NAME"]
    country = df.loc[0, "COUNTRY"]
        
    # Get linkedinbio
    df_to_update.loc[_, "jobtitle"] = jobtitle
    df_to_update.loc[_, "industry"] = industry
    df_to_update.loc[_, "country"] = country
    
df_to_update

# Output

### Update jobtitle, country, industry in Hubspot

In [ ]:
for _, row in df_to_update.iterrows():
    # Init data
    data = {}
    
    # Get data
    hs_object_id = row.hs_object_id
    jobtitle = row.jobtitle
    industry = row.industry
    country = row.country

    # Update 
    if jobtitle != None:
        data = {"properties": 
                {"jobtitle": jobtitle,
                 "industry": industry,
                 "country": country}
               }
    hs.contacts.patch(hs_object_id, data)